# Denison CS181/DA210 Homework 5.c - Step 1

Before you turn this problem in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

In [3]:
import os
import os.path
import sys
import importlib
import io
import pandas as pd
from lxml import etree

if os.path.isdir(os.path.join("../../..", "modules")):
    module_dir = os.path.join("../../..", "modules")
else:
    module_dir = os.path.join("../..", "modules")

module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

import util
importlib.reload(util)

<module 'util' from 'C:\\Users\\ngana\\Documents\\SPRING2022\\CS181\\cs181-s22\\modules\\util.py'>

---

## Part A: String Data and `io.StringIO()`

**Q1:** Consider the following single Python string, `s`, composed over multiple continued lines:

    s = "Twilight and evening bell,\n" \
        "And after that the dark!\n" \
        "And may there be no sadness of farewell,\n" \
        "When I embark;\n"

First, write some code to deal with `s` as a string:

- determine the length of `s`, assign to `len_s`
- find the integer start and end indices (inclusive) of the substring `"dark"` within `s`, and assign to `dark_start`/`dark_end`
- create string `s2` by replacing `"embark"` with `"disembark"`

In [4]:
# Do not change this:
s = "Twilight and evening bell,\n" \
    "And after that the dark!\n" \
    "And may there be no sadness of farewell,\n" \
    "When I embark;\n"

In [20]:
len_s = len(s)
dark_start = s.find('dark')
dark_end = dark_start + 3
s2 = s[:-8] + "disembark" + s[-2:]

In [21]:
# Debugging cell
print(len_s)
print("start:", dark_start, "end:", dark_end, "substring:", s[dark_start:dark_end+1])
print("length s2:", len(s2))

108
start: 46 end: 49 substring: dark
length s2: 111


In [22]:
# Testing cell
assert len_s == 108
assert s[dark_start:dark_end+1] == "dark"
assert len(s2) == 111
assert " embark" not in s2
assert "disembark" in s2

**Q2:** The purpose of `io.StringIO()` is to create a file-like object from *any* string in a Python program.  The object created "acts" just like a file object obtained from an `open()` file would.

Now, create a file-like object from `s` (the same multi-line string from the previous question), and perform a first `readline()`, assigning to variable `line1` and then write a `for` loop or list comprehension to use the file-like object as an iterator to accumulate into a list called `lines` a list of the remaining lines.  For each of the strings in `line1` and `lines`, make sure you omit any trailing newline.

In [33]:
s_file = io.StringIO(s)
line1 = s_file.readline().strip()
lines = [line.strip() for line in s_file.readlines()]

In [34]:
# Debugging cell
print(line1)
print(lines)

print(len(line1))

Twilight and evening bell,
['And after that the dark!', 'And may there be no sadness of farewell,', 'When I embark;']
26


In [35]:
# Testing cell
assert len(line1) == 26
assert "\n" not in line1
assert len(lines) == 3
assert "\n" not in lines[0]
assert lines[-1] == "When I embark;"

---

## Part B: File from the Network

The next set of exercises involve a file at resource path `/data/mystery3.dat` on host `datasystems.denison.edu`.  You can assume the file is textual, and is a tab-separated data collection where each line consists of:
```
    male_name <tab> male_count <tab> female_name <tab> female_count
```
for the top 10 name applications of each sex to the US Social Security Administration for the year 2015.

For example, one line may be:

```
    Noah\t19635\tEmma\t20455\n
```

**Q3:** Write code to:

- acquire the file from the web server
- assign to `status_code` the status_code, and ensure that it is 200

In [92]:
import requests

def buildURL(location, resource, protocol='http'):
    temp = location + resource
    if location[-1] != '/' and resource[0] != '/':
        temp = location + '/' + resource
    url = '{}{}'.format(protocol + "://", temp)
    return url

resource = "/data/mystery3.dat"
location = "datasystems.denison.edu"

url = buildURL(location, resource)
response = requests.get(url)
request = response.request

status_code = response.status_code

In [93]:
# Testing cell
assert status_code == 200

**Q4:** Suppose the exact encoding of the file `mystery3.dat` is unknown, but it is one of the following:

- 'UTF-8'
- 'UTF-16BE'
- 'UTF-16LE'
- 'cp037'
- 'latin_1'

Write code to:

- determine the *correct* encoding and assign to `real_encoding`
- set the `.encoding` attribute of the response to `real_encoding`
- assign to `tsv_body` the string text for the body of the response.

In [99]:
real_encoding = 'cp037'
response.encoding = real_encoding

tsv_body = response.text

print(tsv_body)

Noah	19635	Emma	20455
Liam	18374	Olivia	19691
Mason	16627	Sophia	17417
Jacob	15949	Ava	16378
William	15909	Isabella	15617
Ethan	15077	Mia	14905
James	14824	Abigail	12401
Alexander	14547	Emily	11786
Michael	14431	Charlotte	11398
Benjamin	13700	Harper	10295



In [100]:
# Testing cell
assert "Noah" in tsv_body

**Q5:** In this question, you will start with a `string` and create a `DoL` representation of the data entailed in the string.

It is suggested to use the result of the previous problem, `tsv_body` as the starting point.  But to start independently, you can use the following string literal constant assignment to get to the same starting point:
```
    tsv_body = "Noah\t19635\tEmma\t20455\n" \
               "Liam\t18374\tOlivia\t19691\n" \
               "Mason\t16627\tSophia\t17417\n" \
               "Jacob\t15949\tAva\t16378\n" \
               "William\t15909\tIsabella\t15617\n" \
               "Ethan\t15077\tMia\t14905\n" \
               "James\t14824\tAbigail\t12401\n" \
               "Alexander\t14547\tEmily\t11786\n" \
               "Michael\t14431\tCharlotte\t11398\n" \
               "Benjamin\t13700\tHarper\t10295\n"
```
Construct a file-like object from `tsv_body` and then use file object operations to create a dictionary of lists representation of the tab-separated data.  Note that there is no header line in the data, so you can name the columns `malename`, `malecount`, `femalename`, `femalecount`.

In [50]:
tsv_body = "Noah\t19635\tEmma\t20455\n" \
               "Liam\t18374\tOlivia\t19691\n" \
               "Mason\t16627\tSophia\t17417\n" \
               "Jacob\t15949\tAva\t16378\n" \
               "William\t15909\tIsabella\t15617\n" \
               "Ethan\t15077\tMia\t14905\n" \
               "James\t14824\tAbigail\t12401\n" \
               "Alexander\t14547\tEmily\t11786\n" \
               "Michael\t14431\tCharlotte\t11398\n" \
               "Benjamin\t13700\tHarper\t10295\n"

tsv_file = io.StringIO(tsv_body)
tsv_lines = [line.strip().split('\t') for line in tsv_file]
DoL = {'malename': [item[0] for item in tsv_lines],
       'malecount': [item[1] for item in tsv_lines],
       'femalename': [item[2] for item in tsv_lines],
       'femalecount': [item[3] for item in tsv_lines]}

In [51]:
# Testing cell
assert len(DoL['malename']) == 10
assert DoL['malename'][0] == 'Noah'
assert DoL['femalename'][9] == 'Harper'

**Q6:** Use `pandas` to obtain a data frame named `df` by using a file-like object based on `tsv_body` and use `read_csv()`.  Name your resultant data frame `df`.  Make sure you have reasonable column names.

Be careful to call `read_csv` so that the separators are tabs, not commas.

In [54]:
df = pd.read_csv(io.StringIO(tsv_body), delimiter='\t', names = ['malename', 'malecount', 'femalename', 'femalecount'])

df

,malename,malecount,femalename,femalecount
0,Noah,19635,Emma,20455
1,Liam,18374,Olivia,19691
2,Mason,16627,Sophia,17417
3,Jacob,15949,Ava,16378
4,William,15909,Isabella,15617
5,Ethan,15077,Mia,14905
6,James,14824,Abigail,12401
7,Alexander,14547,Emily,11786
8,Michael,14431,Charlotte,11398
9,Benjamin,13700,Harper,10295


In [55]:
# Testing cell
assert len(df) == 10
assert isinstance(df.columns[0],str)
assert df.iloc[0,0] == 'Noah'
assert df.iloc[0,1] == 19635
assert df.iloc[9,2] == 'Harper'
assert df.iloc[9,3] == 10295

---

## Part C: XML Data from the Network

In many of the following exercises, your task will be to use the Python `requests` module to make an HTTP request, and to obtain the *parsed* XML-based `ElementTree` structure from the result, assigning to variable `root` the root of the result.

You must **always** check the status code returned from the request before further processing.  In some cases, we will ask for a specific method from among those demonstrated in the textbook section.

**Q7:** Using any method, get the XML data from `school0.xml`:
```
    https://datasystems.denison.edu/data/school0.xml
```

In [62]:
resource = "/data/school0.xml"
location = "datasystems.denison.edu"

url = buildURL(location, resource)
response = requests.get(url)
request = response.request

status_code = response.status_code
assert status_code == 200

tree = etree.parse(io.BytesIO(response.content))
root = tree.getroot()

util.print_xml(root, nlines=12)

<school>
  <departments>
    <department id="CINE">
      <name>Cinema</name>
      <division>Fine Arts</division>
      <chair id="9042">
        <lastname>Rice</lastname>
        <firstname>Theresa</firstname>
      </chair>
    </department>
    <department id="MATH">
      <name>Mathematics &amp; Computer Science</name>


In [63]:
# Testing cell
assert root.tag == "school"
assert len(root) == 3

**Q8:** Using the bytes data in `.content`, a *file-like-object*, and `etree.parse()`, get the XML data from `school0.xml`.

In [64]:
tree = etree.parse(io.BytesIO(response.content))
root = tree.getroot()

util.print_xml(root, nlines=12)

<school>
  <departments>
    <department id="CINE">
      <name>Cinema</name>
      <division>Fine Arts</division>
      <chair id="9042">
        <lastname>Rice</lastname>
        <firstname>Theresa</firstname>
      </chair>
    </department>
    <department id="MATH">
      <name>Mathematics &amp; Computer Science</name>


In [65]:
# Testing cell
assert root.tag == "school"
assert len(root) == 3

**Q9:** The `school0_32.xml` resource is encoded with `utf-32be`.  Use the method of setting the `.encoding` attribute of the response and then accessing the `.text` string body, and using `fromstring()`.

Note that `fromstring()` expects to start from an Element, not from the header line, so you will need to skip the header to get the string to pass.

In [71]:
resource = "/data/school0_32.xml"
location = "datasystems.denison.edu"

url = buildURL(location, resource)
response = requests.get(url)
request = response.request

status_code = response.status_code
assert status_code == 200

response.encoding = 'utf-32be'

root = etree.fromstring(response.content)


util.print_xml(root, nlines=12)

<school>
  <departments>
    <department id="CINE">
      <name>Cinema</name>
      <division>Fine Arts</division>
      <chair id="9042">
        <lastname>Rice</lastname>
        <firstname>Theresa</firstname>
      </chair>
    </department>
    <department id="MATH">
      <name>Mathematics &amp; Computer Science</name>


In [72]:
# Testing cell
assert root.tag == "school"
assert len(root) == 3

---

---

## Part D

**Q10:** How much time (in minutes/hours) did you spend on this homework assignment?

1 hour

**Q11:** Who was your partner for this assignment?  If you worked alone, say so instead.

I worked alone